In [ ]:
import pandas as pd
import numpy as np
import sparkge as sg
from sparkge import context as ctx
from sparkge import evolution as evo
from sparkge.evolution import length_of
from sparkge.providers.fitness import *
import warnings
%matplotlib inline
res = np.seterr(divide='ignore', invalid='ignore') #be careful
warnings.filterwarnings("ignore")
#set recusiion limit sparge.configure()

In [119]:
arr = np.ones((100,20), np.int)
def f (data) : return np.random.randint(0,100,len(data)) 
sc =  sparkge_context() 
res = sc.apply_function(arr, f)

In [120]:
res

array([[ 0, 10],
       [ 1, 48],
       [ 2, 81],
       [ 3, 46],
       [ 4,  9],
       [ 5, 47],
       [ 6, 28],
       [ 7, 50],
       [ 8,  6],
       [ 9, 78],
       [10, 79],
       [11, 55],
       [12, 62],
       [13, 66],
       [14, 76],
       [15, 85],
       [16, 10],
       [17, 58],
       [18, 17],
       [19, 96],
       [20, 55],
       [21, 85],
       [22, 89],
       [23, 30],
       [24, 71],
       [25, 41],
       [26, 10],
       [27, 45],
       [28, 51],
       [29, 20],
       [30, 95],
       [31, 17],
       [32, 67],
       [33, 12],
       [34, 71],
       [35,  3],
       [36, 35],
       [37, 68],
       [38, 32],
       [39,  3],
       [40, 48],
       [41, 60],
       [42, 16],
       [43, 39],
       [44, 76],
       [45, 33],
       [46, 77],
       [47, 73],
       [48, 26],
       [49, 46],
       [50, 10],
       [51, 48],
       [52, 81],
       [53, 46],
       [54,  9],
       [55, 47],
       [56, 28],
       [57, 50],
       [58,  6

This notebook describes basic useage of sparkge. See the concepts notebook for a better understanding of the patterns used

# Grammar

In [ ]:
from sparkge.symbols import *

def squared(x): return np.power(x,2)
def cubed (x): return np.power(x,3)

@terminal
def var():
    return ["X"]
    #return ["X", "A", "B", "C"]

@terminal
def param():
    #return ["X"]
    return [ "A", "B"]

@terminal(is_constant=True)
def const():return [ 2,3,4,5]#, np.pi

@operator()
def uop(): return [np.exp, np.log]#np.sin, np.cos , np.log, np.sin,

@operator()
def op(): return [add, sub, mul, truediv]#truediv

@operator()
def power(): return [np.power]

def expr(genome=None, args = []):  
    return as_expression(genome, [ op(expr,expr), power(expr,param) ,var, param])
    #return as_expression(genome,  [op(expr,expr),op(expr,expr),  var])

# Sample Expressions

In [ ]:
power_sample = np.array([1,0,2])
power_law_sample = np.array([0,2,3,1, 1, 1,2])
#sample using a random genome - re-run cell to see other instances
f= expr(evo.chain(l=power_law_sample,max_wraps=2))#np.array([0,0,3,3])
print(f(X=3,A=4,B=6,C=5))
from sparkge.symbols import _repr_
print(_repr_(f))
S= display(f)
S

## Get some sample data

In [ ]:
df = pd.read_csv("./sparkge/data/asm_sample_data_duration.csv")
df.L = df.L.astype(int)
M = df[["L", "M1"]].set_index("L")
SAMPLE = M.sample(5000).sort_index()

# Fit the data to sample expression

In [ ]:
print(display(f))    
_f = accepting_ordered_params(f) #see comments in fnuction accepting_ordered_params
est = curve_fit(_f, SAMPLE, estimate=[[0.1,2,1,]], display=True)
print("Est params", est[0])
print("GoF", goodness_of_fit(_f,SAMPLE, est))

when fitting functions with scipy the convention is to pass the variables parameters in an ordered list f(X, params). To respect that we try to unravel our strict function of f(X=,A=,B=) into something that takes an ordered list. It is important of course to make sure that we map the right ordinal to the right key

# Strategy

In [ ]:
s = evo.strategy(pop_size=100, genome_size=100, srate=0.52, mrate=0.01)

# Evolve a test function

In [ ]:
lengths, fitnesses, genes = s.init()
for generation in range(1000): 
    fitnesses = sg.symbols._evaluate(genes,expr)#default test function
    meta = np.stack([length_of(genes),fitnesses],axis=1)
    genes = s.select_and_mutate(meta,genes)
    sg.store.update(meta,genes)

# Curve fitting data


In [ ]:
lengths, fitnesses, genes = s.init()
for generation in range(1000):   
    fitnesses = sg.symbols._evaluate_against_data(genes,expr,SAMPLE,p0=[[0.1, 2, 1, 1]]) 
    meta = np.stack([length_of(genes),fitnesses],axis=1)
    sg.store.update(meta,genes)
    genes = s.select_and_mutate(meta,genes)

# Spark

In [ ]:
#get a function that broadcasts the stuff and then gives back a function that can be sent to workers
def par_eval(pop):
    #ctx.sc
    #organise ids into mini mbatches and then remotely load data and eval each
    pass

lengths, fitnesses, genes = s.init()
for generation in range(1000):   
    fitnesses = par_eval(genes,expr) 
    meta = np.stack([length_of(genes),fitnesses],axis=1)
    store.update(meta,genes)
    genes = s.select_and_mutate(meta,genes)